In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [2]:
final_dataset = pd.read_csv("./aggregated_dataset.csv")

final_dataset.Label.value_counts()

NameError: name 'pd' is not defined

In [ ]:
final_dataset.shape

(1061600, 9)

In [5]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

print('No of Features: {}'.format(len(features)))

seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

No of Features: 561


561

In [6]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)

# # add subject column to the dataframe
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None, squeeze=True)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'], squeeze=True)
y_train_labels = y_train.map({1: 'WALKING', 2:'WALKING_UPSTAIRS',3:'WALKING_DOWNSTAIRS',\
                       4:'SITTING', 5:'STANDING',6:'LAYING'})

# # put all columns in a single dataframe
train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

C:\Users\ASK\AppData\Local\Temp\ipykernel_38772\1588319156.py:4: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None, squeeze=True)
C:\Users\ASK\AppData\Local\Temp\ipykernel_38772\1588319156.py:6: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'], squeeze=True)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
5804,0.202351,-0.021717,-0.111851,-0.332095,-0.114114,-0.309439,-0.382582,-0.161981,-0.308824,0.046627,...,0.831735,0.495929,-0.021844,-0.621611,-0.843964,0.187828,-0.04474,26,1,WALKING


In [10]:
# get the data from txt files to pandas dataffame
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)

# add subject column to the dataframe
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None, squeeze=True)

# get y labels from the txt file
y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'], squeeze=True)
y_test_labels = y_test.map({1: 'WALKING', 2:'WALKING_UPSTAIRS',3:'WALKING_DOWNSTAIRS',\
                       4:'SITTING', 5:'STANDING',6:'LAYING'})


# put all columns in a single dataframe
test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

C:\Users\ASK\AppData\Local\Temp\ipykernel_38772\1155623851.py:5: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None, squeeze=True)
C:\Users\ASK\AppData\Local\Temp\ipykernel_38772\1155623851.py:8: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'], squeeze=True)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
1766,0.271549,-0.016906,-0.11077,-0.996921,-0.989715,-0.986132,-0.9973,-0.988802,-0.984284,-0.941987,...,0.162022,-0.701201,0.410483,-0.567081,0.347885,-0.774571,-0.142021,13,6,LAYING


# Train

In [15]:
train.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler


In [17]:
from ELM import ML_FELM, ELMClassifier,FELM_AE

## HAR Dataset

In [51]:
from sklearn.preprocessing import LabelEncoder
X_train = train.drop(['subject', 'Activity', 'ActivityName'], axis=1).to_numpy()
X_test = test.drop(['subject', 'Activity', 'ActivityName'], axis=1).to_numpy()
y_train = train['Activity'].to_numpy()
y_test = test['Activity'].to_numpy()

y_train = y_train - 1
y_test = y_test - 1

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [28]:
print((len(X_train)))
print(len(y_train))
print(len(X_test))
print(len(y_test))

7352
7352
2947
2947


In [55]:
import time
from sklearn.metrics import accuracy_score, classification_report

# ELM Classifier
print("ELM")
start_time = time.time()
elm = ELMClassifier(n_hidden=256, activation="relu")
elm.fit(X_train, y_train)
end_time = time.time()

# Predict and evaluate
y_pred = elm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Print results
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy:.2f}")
print(f"ELM Training Time: {end_time - start_time:.2f} seconds")

print("\n============================")


ELM
              precision    recall  f1-score   support

           0       0.89      0.99      0.93       496
           1       0.91      0.90      0.90       471
           2       0.98      0.86      0.92       420
           3       0.90      0.82      0.86       491
           4       0.82      0.92      0.86       532
           5       1.00      0.95      0.98       537

    accuracy                           0.91      2947
   macro avg       0.92      0.91      0.91      2947
weighted avg       0.91      0.91      0.91      2947

Accuracy: 0.91
ELM Training Time: 0.37 seconds



In [111]:
# ML-FELM Classifier
print("ML-FELM")
layer_sizes = [256, 128]  # Number of hidden nodes in each layer
fisher_lambdas = [0.5, 0.1]  # Fisher regularization for each layer

start_time = time.time()
ml_felm = ML_FELM(layer_sizes, fisher_lambdas, activation="relu")
ml_felm.fit(X_train, y_train)
end_time = time.time()

# Predict and evaluate
y_pred = ml_felm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Print results
print(classification_report(y_test, y_pred))
print(f"Test Accuracy: {accuracy:.2f}")
print(f"ML-FELM Training Time: {end_time - start_time:.2f} seconds")

ML-FELM
              precision    recall  f1-score   support

           0       0.97      0.96      0.97       496
           1       0.94      0.95      0.94       471
           2       0.92      0.93      0.93       420
           3       0.89      0.86      0.88       491
           4       0.88      0.91      0.90       532
           5       1.00      1.00      1.00       537

    accuracy                           0.94      2947
   macro avg       0.93      0.93      0.93      2947
weighted avg       0.94      0.94      0.94      2947

Test Accuracy: 0.94
ML-FELM Training Time: 1.62 seconds


In [57]:
import pygad
import numpy as np
from sklearn.metrics import f1_score

## GA ML-FELM

In [94]:
def fitness_function(ga_instance, solution, solution_idx):
    feature_genes = solution[:-2]
    k = [int(solution[-2])]  # Third-to-last gene is the value of k
    fisher = [solution[-1]]  # Last gene (not used in computation)

    feature_genes = np.rint(feature_genes)
    
    selected_features = np.where(feature_genes == 1)[0]
    
    if len(selected_features) == 0 or k[0] < 1:
        return 0
    
    # Subset the dataset
    X_train_subset = X_train[:, selected_features]
    X_test_subset = X_test[:, selected_features]
    
    # Train the classifier
    model = ML_FELM(k, fisher, activation="relu")
    model.fit(X_train_subset, y_train)
    
    # Predict and calculate the F1-score
    y_pred = model.predict(X_test_subset)
    f1 = f1_score(y_test, y_pred, average="weighted")  # Weighted for multi-class
    return f1

In [95]:
# GA parameters
num_features = X_train.shape[1]
num_generations = 50
num_parents_mating = 5
sol_per_pop = 8
num_genes = num_features + 2  # Features + k + fisher

# Define gene space
gene_space = [
    {"low": 0, "high": 1},  # Feature selection genes (binary)
] * num_features + [{"low": 1, "high": 256, "step": 1},  # k values (range 1 to 20 () integer)
                    {"low": 0.0, "high": 1.0}]  # fisher

In [96]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    fitness_func=fitness_function,
    sol_per_pop=sol_per_pop,
    num_genes=num_genes,
    gene_space=gene_space,
    gene_type=float,
    parent_selection_type="sss",
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=10,
    stop_criteria=["reach_1"]
)

# Run the GA
ga_instance.run()

# Best solution
best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
best_feature = best_solution[:-2]
best_feature = np.rint(best_feature)
best_selected_features = np.where(best_feature == 1)[0]

best_hidden_size = best_solution[-2]

best_fisher = best_solution[-1]

print("Best Selected Features : ", best_selected_features)
print(len(best_selected_features))

print("Best Hidden Size : ", best_hidden_size)

print("Best Fisher : ", best_fisher)

print(f"Best solution fitness (F1-score): {best_solution_fitness}")


c:\Users\ASK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Best Selected Features :  [  1   4   6   8   9  10  11  15  18  20  23  33  39  41  43  44  47  49
  51  55  56  57  62  63  68  72  75  78  86  87  91  93  94  97  99 102
 104 106 110 112 114 115 116 118 119 124 125 133 135 137 138 139 141 142
 143 144 149 151 152 154 155 156 157 158 162 163 165 178 179 180 181 183
 184 185 187 189 191 194 196 197 198 200 202 203 204 205 211 212 215 216
 221 222 224 226 227 228 229 231 235 238 239 241 246 249 251 255 257 261
 264 266 271 278 279 280 283 294 295 296 297 298 299 302 308 312 313 317
 320 323 325 327 328 329 330 331 332 334 337 341 342 343 345 348 353 354
 355 358 361 365 366 371 373 375 380 381 382 386 391 392 394 396 398 401
 404 407 411 412 413 415 416 419 426 427 429 430 433 434 437 438 439 441
 445 447 448 454 456 460 465 466 467 468 470 472 474 476 478 479 481 486
 487 488 492 493 496 499 503 504 505 506 508 509 510 511 513 514 516 519
 520 521 522 524 525 526 532 535 537 543 546 551 552 554]
230
Best Hidden Size :  252.0
Best Fishe

## 2 Layer ML-FELM

In [117]:
def fitness_function(ga_instance, solution, solution_idx):
    feature_genes = solution[:-4]
    k = [int(solution[-4]), int(solution[-3])]
    fisher = [solution[-2], solution[-1]]

    feature_genes = np.rint(feature_genes)
    
    selected_features = np.where(feature_genes == 1)[0]
    
    if len(selected_features) == 0 or np.any(np.array(k) < 1):
        return 0
    
    # Subset the dataset
    X_train_subset = X_train[:, selected_features]
    X_test_subset = X_test[:, selected_features]
    
    # Train the classifier
    model = ML_FELM(k, fisher, activation="relu")
    model.fit(X_train_subset, y_train)
    
    # Predict and calculate the F1-score
    y_pred = model.predict(X_test_subset)
    f1 = f1_score(y_test, y_pred, average="weighted")  # Weighted for multi-class
    return f1

In [118]:
# GA parameters
num_features = X_train.shape[1]
num_generations = 50
num_parents_mating = 5
sol_per_pop = 8
num_genes = num_features + 4  # Features + k + fisher

gene_space = (
    [{"low": 0, "high": 1}] * num_features +  # Features (binary genes: 0 or 1)
    [{"low": 1, "high": 256, "step": 1}] * 2 +  # k values (integer range: 1 to 256)
    [{"low": 0.0, "high": 1.0}] * 2  # Fisher (float range: 0.0 to 1.0)
)

In [119]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    fitness_func=fitness_function,
    sol_per_pop=sol_per_pop,
    num_genes=num_genes,
    gene_space=gene_space,
    gene_type=float,
    parent_selection_type="sss",
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=10,
    stop_criteria=["reach_1"]
)

# Run the GA
ga_instance.run()

# Best solution
best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
best_feature = best_solution[:-4]
best_feature = np.rint(best_feature)
best_selected_features = np.where(best_feature == 1)[0]

best_hidden_size = best_solution[-4:-2]

best_fisher = best_solution[-2:]

print("Best Selected Features : ", best_selected_features)
print(len(best_selected_features))

print("Best Hidden Size : ", best_hidden_size)

print("Best Fisher : ", best_fisher)

print(f"Best solution fitness (F1-score): {best_solution_fitness}")


c:\Users\ASK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Best Selected Features :  [  0   4   5   6  12  15  16  18  21  22  23  24  25  27  28  36  37  41
  50  51  52  54  55  62  63  65  67  68  69  70  73  76  82  83  84  85
  87  88  91  92  93  95  97  99 100 101 102 103 104 108 109 114 120 121
 122 126 127 128 129 131 132 133 137 141 142 144 145 146 147 149 150 153
 155 160 165 166 171 175 176 177 178 179 180 181 182 183 184 186 188 195
 196 197 198 201 203 207 208 209 211 213 214 216 221 234 236 239 240 241
 242 243 247 250 253 255 266 267 272 273 275 276 277 278 282 283 285 287
 288 290 291 293 294 296 297 302 303 306 308 311 312 316 317 318 321 322
 323 325 326 327 329 332 333 334 335 340 342 346 347 348 349 350 351 355
 356 358 359 362 364 370 371 372 374 376 381 384 386 387 390 393 394 395
 396 402 403 405 409 410 414 415 419 420 422 424 425 426 427 428 430 431
 433 434 437 439 444 446 447 448 450 451 454 455 456 457 460 461 462 467
 468 471 472 474 475 477 487 488 490 492 502 503 504 511 512 520 525 527
 529 530 534 536 539 545 

## GA ELM

In [90]:
def fitness_function(ga_instance, solution, solution_idx):
    feature_genes = solution[:-1]
    k = int(solution[-1])  # Third-to-last gene is the value of k

    feature_genes = np.rint(feature_genes)
    
    selected_features = np.where(feature_genes == 1)[0]
    
    if len(selected_features) == 0 or k < 1:
        return 0
    
    # Subset the dataset
    X_train_subset = X_train[:, selected_features]
    X_test_subset = X_test[:, selected_features]
    
    # Train the classifier
    model = ELMClassifier(n_hidden=k, activation="relu")
    model.fit(X_train_subset, y_train)
    
    # Predict and calculate the F1-score
    y_pred = model.predict(X_test_subset)
    f1 = f1_score(y_test, y_pred, average="weighted")  # Weighted for multi-class
    return f1

In [91]:
# GA parameters
num_features = X_train.shape[1]
num_generations = 50
num_parents_mating = 5
sol_per_pop = 8
num_genes = num_features + 1  # Features + k

# Define gene space
gene_space = [
    {"low": 0, "high": 1},  # Feature selection genes (binary)
] * num_features + [{"low": 1, "high": 256, "step": 1}]  # k values (range 1 to 20 () integer) 

In [93]:
ga_instance = pygad.GA(
    num_generations=num_generations,
    num_parents_mating=num_parents_mating,
    fitness_func=fitness_function,
    sol_per_pop=sol_per_pop,
    num_genes=num_genes,
    gene_space=gene_space,
    gene_type=float,
    parent_selection_type="sss",
    crossover_type="single_point",
    mutation_type="random",
    mutation_percent_genes=10,
    stop_criteria=["reach_1"]
)

# Run the GA
ga_instance.run()

# Best solution
best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
best_feature = best_solution[:-1]
best_feature = np.rint(best_feature)
best_selected_features = np.where(best_feature == 1)[0]

best_hidden_size = best_solution[-1]

print("Best Selected Features : ", best_selected_features)
print(len(best_selected_features))

print("Best Hidden Size : ", best_hidden_size)

print(f"Best solution fitness (F1-score): {best_solution_fitness}")


c:\Users\ASK\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


Best Selected Features :  [  1   3   8  10  11  12  15  19  20  21  25  26  29  38  39  42  46  47
  48  53  56  63  71  72  73  75  78  83  84  85  87  90  92  93  94  95
  96  98 100 101 102 105 106 107 111 121 122 126 128 129 130 133 135 138
 139 140 142 144 145 146 149 152 156 157 165 169 172 173 176 178 182 185
 186 187 188 193 195 197 200 201 202 203 205 207 209 211 213 214 215 216
 217 218 223 225 226 228 232 234 237 238 240 241 242 244 249 251 253 259
 260 262 265 266 273 275 280 281 285 294 297 299 302 310 312 323 330 333
 334 336 340 343 344 345 347 348 349 354 355 356 358 360 365 368 374 376
 377 379 380 384 385 388 391 394 396 398 399 400 401 403 405 407 411 412
 413 416 418 419 424 425 426 427 430 431 433 434 438 439 442 446 448 451
 452 456 458 459 461 462 465 466 476 477 478 481 483 486 489 490 492 497
 499 501 504 505 506 508 510 511 513 515 518 521 526 529 530 533 536 537
 541 542 543 546 550 551 554 556 557 558]
226
Best Hidden Size :  240.0
Best solution fitness (F1-